In [189]:
from random import random

import math

# Collect data

In [10]:
def loadMovieLens(path='data/movielens'):
    #Get movie titles
    movies={}
    for line in open(path+'/u.item'):
        id,title=line.split('|')[0:2]
        movies[id]=title

    # Load data
    prefs={}
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
        
    return prefs

In [57]:
data = loadMovieLens()

## Explore data

In [248]:
data['344']


{'Absolute Power (1997)': 3.0,
 'Addicted to Love (1997)': 3.0,
 'Aladdin (1992)': 4.0,
 'Alien (1979)': 5.0,
 'Aliens (1986)': 5.0,
 'Amadeus (1984)': 5.0,
 "Antonia's Line (1995)": 4.0,
 'Apt Pupil (1998)': 5.0,
 'As Good As It Gets (1997)': 4.0,
 'Babe (1995)': 5.0,
 'Back to the Future (1985)': 4.0,
 'Beautiful Girls (1996)': 3.0,
 'Beautiful Thing (1996)': 3.0,
 'Beauty and the Beast (1991)': 5.0,
 'Bed of Roses (1996)': 3.0,
 'Before Sunrise (1995)': 4.0,
 'Being There (1979)': 5.0,
 'Big Bully (1996)': 1.0,
 'Big Night (1996)': 5.0,
 'Birdcage, The (1996)': 4.0,
 'Blade Runner (1982)': 5.0,
 'Bound (1996)': 4.0,
 'Braveheart (1995)': 3.0,
 'Brazil (1985)': 5.0,
 'Breakdown (1997)': 3.0,
 'Broken Arrow (1996)': 3.0,
 'Brothers McMullen, The (1995)': 3.0,
 'Cable Guy, The (1996)': 1.0,
 'Chasing Amy (1997)': 4.0,
 'City Hall (1996)': 3.0,
 'Close Shave, A (1995)': 5.0,
 'Clueless (1995)': 5.0,
 'Contact (1997)': 3.0,
 'Copycat (1995)': 3.0,
 'Courage Under Fire (1996)': 3.0,
 'Cra

# Creation of train set and test set
We want to split data in two set (train and test)

Actually : 

 ** train= 80%totaldataset**
 
 ** test = 20%totaldataset**

In [78]:
def split_train_test(data,percent_test):
    test={}
    train={}
    movie={}
    for u in data.keys():
        test.setdefault(u,{})
        train.setdefault(u,{})
        for movie in data[u]:
            #print(data[u][movie])
            if (random()<percent_test):
                test[u][movie]=data[u][movie]
            else:
                train[u][movie]=data[u][movie]
    return train, test

In [ ]:
percent_test=0.2
train,test=split_train_test(data,percent_test)

## Part that allows to clean train and test
We don't want to have user in test set which are not in train test, the same for the movies so we delete them

In [249]:

def get_moove(data):
    moove = {}
    for u in data:
        for m in data[u]:
            moove[m]=0
    return moove
            
def get_youser(data):
    youser = {}
    for u in data:
        youser[u]=0
    return youser  

def clean(d1,d2):
    to_erase = {}
    for i in d1:
        try:
            d2[i]
        except KeyError:
            to_erase[i]=0
    for i in d2:
        try:
            d1[i]
        except KeyError:
            to_erase[i]=0
    return to_erase
def _remove_users(test,rem):
    for i in rem:
        try:
            del test[i]
        except KeyError:
            pass
def _remove_movies(test,rem):
    for i in test:
        for j in rem:
            try:
                del test[i][j]
            except KeyError:
                pass


In [121]:
mooveToRemoove = clean(get_moove(train),get_moove(test))
youserToRemoove = clean(get_youser(train),get_youser(test))
_remove_users(test,youserToRemoove)
_remove_movies(test,mooveToRemoove)

1384

# Collaboritive Filtering classes

In [244]:
class BaselineMeanUser:
    def __init__(self):
        self.users={}
        self.movies={}
    def fit(self,train):
        for user in train:
            note=0
            for movie in train[user]:
                note+=train[user][movie]
            note=note/len(train[user])
            self.users[user]=round(note)
        
    def predict(self,user,movie):
        return self.users[user]
    def score(self,X):
        nb_movies = len(get_moove(X))
        score = 0
        for user in X:
            for movie in X[user]:
                if(self.predict(user,movie)==X[user][movie]):
                    score+=1
        return float(score)/nb_movies
                
                    
    
class BaselineMeanMovie:
    def __init__(self):
        self.users={}
        self.movies={}
    def fit(self,train):
        movies = get_moove(train)
        for movie in movies:
            note=0
            cpt=0
            for user in train:
                try:
                    note+=train[user][movie]
                    cpt+=1
                except KeyError:
                    pass
            note=note/cpt
            self.movies[movie]=round(note)
        
    def predict(self,user,movie):
        return self.movies[movie]
    def score(self,X):
        nb_movies = len(get_moove(X))
        score = 0
        for user in X:
            for movie in X[user]:
                if(self.predict(user,movie)==X[user][movie]):
                    score+=1
        return float(score)/nb_movies

In [245]:
baseline_mu= BaselineMeanUser()
baseline_mm= BaselineMeanMovie()

In [246]:
baseline_mu.fit(train)
baseline_mm.fit(train)

In [247]:
print("score baseline mean user  ",baseline_mu.score(test))
print("score baseline mean movie ",baseline_mm.score(test))

('score baseline mean user  ', 5.195086705202312)
('score baseline mean movie ', 5.277456647398844)
